In [1]:
# M5 forecasting for the Kaggle competition
# Author chenxia
# calendar.csv, sales_train_validation.csv, sample_submission.csv, sell_prices.csv
# 2011-01-29 to 2016-05-22
# Calendat.csv 包括产品销售的日期信息
# sales_train_validation.csv 包括每个产品每天的销售量
# sell_prices.csv 包括每个产品的价格信息
# sample_submission.csv 包括提交的格式

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 日历包括：日期、week id、weekday、month、year、event name、type、second event name、type、是否允许购买CA、TX、WI
#  Supply nutrition assistance program SNAP 是否允许食品券购买
df_calendar = pd.read_csv('../../data/m5-forecasting-accuracy/calendar.csv')
# 30490个产品，每个产品每天的销售量，departure id、分类id、商店id、状态id、日期、销售量
# 总共3090类产品、10个商店、3个州、每个商店的类别
df_sales_train_validation = pd.read_csv('../../data/m5-forecasting-accuracy/sales_train_validation.csv')
# sell prices 包括：商店id、产品id、日期、价格
df_sell_prices = pd.read_csv('../../data/m5-forecasting-accuracy/sell_prices.csv')
df_sample_submission = pd.read_csv('../../data/m5-forecasting-accuracy/sample_submission.csv')

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
print(df_calendar.head())
print(df_sales_train_validation.head())
print(df_sales_train_validation.info())
print(df_sell_prices.head())
print(df_sample_submission.head())

         date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0        1  
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  

In [12]:
# 构建特征工程
# 1. 价格特征
d_cols = [c for c in df_sales_train_validation.columns if 'd_' in c] # sales data columns
x = df_sales_train_validation[d_cols].copy()

target_day = 1914

#使用历史数据中最后的7天构造特征
local_range = 7

# 由于使用前1913天的数据预测第1914天，历史数据与预测目标的距离只有1天，因此predict_distance=1
predict_distance = 1

def get_local_features(target_day, predict_distance):
    local_features = pd.DataFrame()
    for i in range(local_range):
        local_features['l_'+str(i + 1)] = x['d_'+str(target_day - i - predict_distance)].astype(float)
    l_cols = ['l_'+str(i + 1) for i in range(local_range)]
    return local_features[l_cols]

print(get_local_features(target_day,predict_distance))


       l_1  l_2  l_3  l_4  l_5  l_6  l_7
0      1.0  1.0  0.0  3.0  1.0  1.0  1.0
1      0.0  0.0  0.0  0.0  1.0  0.0  0.0
2      1.0  1.0  1.0  0.0  1.0  1.0  1.0
3      2.0  7.0  3.0  1.0  0.0  1.0  4.0
4      4.0  2.0  2.0  2.0  1.0  1.0  0.0
...    ...  ...  ...  ...  ...  ...  ...
30485  1.0  0.0  0.0  1.0  0.0  0.0  0.0
30486  0.0  1.0  0.0  0.0  0.0  0.0  0.0
30487  0.0  1.0  0.0  0.0  1.0  0.0  2.0
30488  3.0  1.0  3.0  0.0  1.0  0.0  0.0
30489  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[30490 rows x 7 columns]


In [13]:
def get_local_accumulated_feature(target_day, predict_distance):
    local_accumulated_feature = pd.DataFrame()
    local_accumulated_feature['la_1'] = x['d_'+str(target_day - predict_distance)].astype(float)
    for i in range(1, local_range):
        local_accumulated_feature['la_'+str(i + 1)] = x['d_'+str(target_day - i - predict_distance)].astype(float) +\
            local_accumulated_feature['la_' + str(i)]
    la_cols = ['la_'+str(i + 1) for i in range(local_range)]
    return local_accumulated_feature[la_cols]

get_local_accumulated_feature(target_day, predict_distance)

,la_1,la_2,la_3,la_4,la_5,la_6,la_7
0,1.0,2.0,2.0,5.0,6.0,7.0,8.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,1.0,2.0,3.0,3.0,4.0,5.0,6.0
3,2.0,9.0,12.0,13.0,13.0,14.0,18.0
4,4.0,6.0,8.0,10.0,11.0,12.0,12.0
...,...,...,...,...,...,...,...
30485,1.0,1.0,1.0,2.0,2.0,2.0,2.0
30486,0.0,1.0,1.0,1.0,1.0,1.0,1.0
30487,0.0,1.0,1.0,1.0,2.0,2.0,4.0
30488,3.0,4.0,7.0,7.0,8.0,8.0,8.0


In [15]:
tx = x[d_cols[::-1]].cumsum(axis=1)
used_history_distances = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 42, 56]

def get_accumulated_features(target_day, predict_distance):
    long_term_features = pd.DataFrame()
    for distance in used_history_distances:
        long_term_features['la_' + str(distance)] = tx['d_' + str(target_day - distance - predict_distance + 1)].astype(float)
    la_cols = ['la_' + str(distance) for distance in used_history_distances]
    return long_term_features[la_cols]

get_accumulated_features(target_day, predict_distance)

,la_1,la_2,la_3,la_4,la_5,la_6,la_7,la_14,la_21,la_28,la_42,la_56
0,1.0,2.0,2.0,5.0,6.0,7.0,8.0,13.0,25.0,27.0,45.0,55.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,11.0
2,1.0,2.0,3.0,3.0,4.0,5.0,6.0,14.0,15.0,16.0,19.0,31.0
3,2.0,9.0,12.0,13.0,13.0,14.0,18.0,29.0,45.0,51.0,90.0,104.0
4,4.0,6.0,8.0,10.0,11.0,12.0,12.0,20.0,24.0,38.0,55.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30485,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,6.0,6.0,6.0,13.0
30486,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,7.0,9.0,9.0,9.0
30487,0.0,1.0,1.0,1.0,2.0,2.0,4.0,11.0,22.0,25.0,37.0,60.0
30488,3.0,4.0,7.0,7.0,8.0,8.0,8.0,13.0,22.0,26.0,45.0,61.0


In [16]:
# 构建周期特征
def get_period_sale(target_day, predict_distance):
    period = 7
    i_start = (predict_distance + period - 1) // period
    period_sale = pd.DataFrame()
    for i in range(8):
        cur_day = target_day - (i + i_start) * period
        period_sale['p_'+str(i + 1)] = x['d_' + str(cur_day)].astype(float)
    return period_sale

get_period_sale(target_day, predict_distance)

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.0,2.0,1.0,0.0,4.0,3.0,3.0,0.0
4,0.0,2.0,0.0,1.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
30485,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
30486,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
30487,2.0,4.0,2.0,1.0,2.0,0.0,1.0,1.0
30488,0.0,0.0,1.0,3.0,1.0,0.0,0.0,2.0


In [17]:
def get_period_features(target_day, predict_distance):
    tx_period = get_period_sale(target_day, predict_distance)
    tx_period = tx_period.cumsum(axis=1)
    return tx_period

get_period_features(target_day, predict_distance)

,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8
0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0
1,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
3,4.0,6.0,7.0,7.0,11.0,14.0,17.0,17.0
4,0.0,2.0,2.0,3.0,5.0,7.0,9.0,10.0
...,...,...,...,...,...,...,...,...
30485,0.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0
30486,0.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0
30487,2.0,6.0,8.0,9.0,11.0,11.0,12.0,13.0
30488,0.0,0.0,1.0,4.0,5.0,5.0,5.0,7.0


In [18]:
def get_history_features(target_day, predict_distance):
    return pd.concat([get_accumulated_features(target_day, predict_distance),
                      get_period_features(target_day, predict_distance)], axis=1)

get_history_features(target_day, predict_distance)

,la_1,la_2,la_3,la_4,la_5,la_6,la_7,la_14,la_21,la_28,la_42,la_56,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8
0,1.0,2.0,2.0,5.0,6.0,7.0,8.0,13.0,25.0,27.0,45.0,55.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,11.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0
2,1.0,2.0,3.0,3.0,4.0,5.0,6.0,14.0,15.0,16.0,19.0,31.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
3,2.0,9.0,12.0,13.0,13.0,14.0,18.0,29.0,45.0,51.0,90.0,104.0,4.0,6.0,7.0,7.0,11.0,14.0,17.0,17.0
4,4.0,6.0,8.0,10.0,11.0,12.0,12.0,20.0,24.0,38.0,55.0,69.0,0.0,2.0,2.0,3.0,5.0,7.0,9.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,6.0,6.0,6.0,13.0,0.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0
30486,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,7.0,9.0,9.0,9.0,0.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0
30487,0.0,1.0,1.0,1.0,2.0,2.0,4.0,11.0,22.0,25.0,37.0,60.0,2.0,6.0,8.0,9.0,11.0,11.0,12.0,13.0
30488,3.0,4.0,7.0,7.0,8.0,8.0,8.0,13.0,22.0,26.0,45.0,61.0,0.0,0.0,1.0,4.0,5.0,5.0,5.0,7.0
